In [ ]:
pip install requests gspread oauth2client beautifulsoup4


In [26]:
import requests
import pandas as pd
import time

# Replace with your actual API key
api_key = "Paste API key here"

# Function to get latitude and longitude from a location name
def get_lat_long(location_name):
    api_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location_name}&key={api_key}"
    response = requests.get(api_url)
    data = response.json()

    if data['status'] == 'OK' and data['results']:
        latitude = data['results'][0]['geometry']['location']['lat']
        longitude = data['results'][0]['geometry']['location']['lng']
        return latitude, longitude
    else:
        print("Failed to get coordinates. Check the location name.")
        return None, None

# Function to get business data from Google Places API
def get_business_data(query, latitude, longitude, max_results=500, radius=5000):
    location_coordinates = f"{latitude},{longitude}"
    businesses = []

    search_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location_coordinates}&radius={radius}&type=business&keyword={query}&key={api_key}'
    search_response = requests.get(search_url)
    data = search_response.json()

    while data.get('results') and len(businesses) < max_results:
        for business in data['results']:
            place_id = business.get('place_id')
            details_url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,formatted_phone_number,website&key={api_key}'
            details_response = requests.get(details_url)
            details_data = details_response.json()

            name = details_data.get('result', {}).get('name', 'N/A')
            phone = details_data.get('result', {}).get('formatted_phone_number', 'N/A')
            website = details_data.get('result', {}).get('website', 'N/A')

            #if phone != 'N/A' and website != 'N/A':  # In case null entries need be rejected
            businesses.append([name, phone, website])

            if len(businesses) >= max_results:
                break

        next_page_token = data.get('next_page_token')
        if next_page_token:
            time.sleep(2)
            search_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location_coordinates}&radius={radius}&type=business&keyword={query}&key={api_key}&pagetoken={next_page_token}'
            search_response = requests.get(search_url)
            data = search_response.json()
        else:
            break

    return businesses

# Get location from user
location_name = input("Enter the location (e.g., New York): ")
latitude, longitude = get_lat_long(location_name)

if latitude and longitude:
    query = input("Enter business type (e.g., Plumber): ")
    max_results = int(input("Enter the number of results you want (up to 500): "))
    businesses = get_business_data(query, latitude, longitude, max_results)

    # Save to DataFrame and print results
    df = pd.DataFrame(businesses, columns=['Name', 'Phone', 'Website'])
    #df.to_csv("Output file path") 
    print("Completed")
else:
    print("Could not retrieve latitude and longitude. Exiting...")


Enter the location (e.g., New York): Delhi
Enter business type (e.g., Plumber): Pharmacy
Enter the number of results you want (up to 500): 50
Completed


In [27]:
df

,Name,Phone,Website
0,SUPER Pharmacy,092123 30118,N/A
1,Apollo Pharmacy Lajpat Nagar,080 4101 8286,https://www.apollopharmacy.in/?utm_source=gmb&...
2,Zedson Pharmacy,098117 31473,http://www.zedsonpharmacy.com/
3,The Pharmacy,098999 62503,N/A
4,Care Pharmacy,098117 27294,N/A
5,Apollo Pharmacy DLF,080 4101 8269,https://www.apollopharmacy.in/?utm_source=gmb&...
6,Apollo Pharmacy Lajpat Nagar 2,080 4101 8142,https://www.apollopharmacy.in/?utm_source=gmb&...
7,Apollo Pharmacy New Patel Nagar,080 4101 8120,https://www.apollopharmacy.in/?utm_source=gmb&...
8,"Apollo Pharmacy - Defence Colony, Delhi",080 4101 8124,https://www.apollopharmacy.in/?utm_source=gmb&...
9,Apollo Pharmacy Old Rajinder Nagar,080 4101 8284,https://www.apollopharmacy.in/?utm_source=gmb&...
